In [52]:

import numpy as np
import pandas as pd

df = pd.read_csv("snow_crab_survey.csv", encoding="windows-1252")
df

,year__annee,month__mois,day__jour,tow_trait,longitude,latitude,start__depart,comment__commentaire,snow_crab_males__males_de_crabe_des_neiges,snow_crab_females__females_de_crabe_des_neiges,Atlantic_cod__morue_franche,American_plaice__plie_americaine,Yellowtail_flounder__limande_a_queue_jaune,winter_flounder__plie_rouge,thorny_skate__raie_epineuset,smooth_skate__raie_lisse,longhorn_sculpin__chaboisseau_a_dix_huit_epines,sea_potato__patate_de_mer,Hyas_coarctatus
0,2019,7,12,GP354F,-60.3894,47.38342,5:12:29,Variable 2 a 2.4 noeuds,0,0,27,0,0,0,1,0,0,0,11
1,2019,7,12,GP351F,-60.4655,47.43065,6:55:00,Stable ~ 2.1 kts,64,1,6,113,0,0,16,2,0,0,0
2,2019,7,12,GP353FR,-60.4038,47.52278,9:50:55,Force a cause de vase ~ 2.1 kts,0,0,0,0,0,0,1,4,0,0,0
3,2019,7,12,GP352F,-60.4394,47.55585,11:10:07,Stable ~ 2.1 kts,18,0,1,0,0,0,1,5,0,0,0
4,2019,7,12,GP350F,-60.4835,47.51007,12:32:42,Stable ~ 2.2 kts,0,0,10,3,0,0,27,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,2021,9,22,GP311F,-61.1144,47.16361,14:54:40,Variable 2 a 2.4 noeuds,24,80,14,293,23,0,0,0,0,0,0
1051,2021,9,22,GP329F,-60.9160,47.16622,16:23:06,Stable ~ 2.2 kts,14,31,25,174,0,0,1,0,0,0,0
1052,2021,9,22,GP333F,-60.8754,47.26900,17:23:09,Variable 2 a 2.4 noeuds,37,3,37,286,1,0,0,0,0,0,0
1053,2021,9,22,GP308F,-61.1538,47.32288,19:31:12,Stable ~ 2.2 kts,52,2,11,250,66,0,1,0,0,0,3


In [56]:
# get a distinct list of years
years = df["year__annee"].unique().tolist()


# add a new column that is a datetime
def combine_date(row):
    year = str(row["year__annee"]).zfill(4)
    month = str(row["month__mois"]).zfill(2)
    day = str(row["day__jour"]).zfill(2)
    time = row["start__depart"]
    if not isinstance(time, str) and np.isnan(time):
        return f"{year}-{month}-{day}"
    else:
        return f"{year}-{month}-{day} {time}"


df["datetime"] = pd.to_datetime(df.apply(combine_date, axis=1))
df_trunc = df.sort_values("datetime").loc[:, ["year__annee", "latitude", "longitude", "datetime"]]
df_trunc

,year__annee,latitude,longitude,datetime
0,2019,47.38342,-60.3894,2019-07-12 05:12:29
1,2019,47.43065,-60.4655,2019-07-12 06:55:00
2,2019,47.52278,-60.4038,2019-07-12 09:50:55
3,2019,47.55585,-60.4394,2019-07-12 11:10:07
4,2019,47.51007,-60.4835,2019-07-12 12:32:42
...,...,...,...,...
1050,2021,47.16361,-61.1144,2021-09-22 14:54:40
1051,2021,47.16622,-60.9160,2021-09-22 16:23:06
1052,2021,47.26900,-60.8754,2021-09-22 17:23:09
1053,2021,47.32288,-61.1538,2021-09-22 19:31:12


In [68]:
import fiona

schema = {
    'geometry': 'LineString',
    'properties': [('Name', 'str')]
}

lineShp = fiona.open('snow_crab_coverage.shp', mode='w', driver='ESRI Shapefile', schema=schema, crs="EPSG:4326")

for year in years:
    mask = df_trunc["year__annee"] == year
    coords = list()
    for index, row in df_trunc[mask].iterrows():
        coords.append((row["longitude"], row["latitude"]))

    rowDict = {
        'geometry': {'type': 'LineString',
                     'coordinates': coords},
        'properties': {'YearAnnee': year},
    }
    lineShp.write(rowDict)

#close fiona object
lineShp.close()


